Данное задание основано на материалах лекций по линейной регрессии и посвящено предсказанию оклада, исходя из описания вакансии.

Вы научитесь:
* использовать линейную регрессию
* применять линейную регрессию к текстовым данным

### Введение

Линейные методы хорошо подходят для работы с разреженными данными — к таковым относятся, например, тексты. Это можно объяснить высокой скоростью обучения и небольшим количеством параметров, благодаря чему удается избежать переобучения.

Линейная регрессия имеет несколько разновидностей в зависимости от того, какой регуляризатор используется. Мы будем работать с гребневой регрессией, где применяется квадратичный, или L2-регуляризатор.

### Реализация в Scikit-Learn

Для извлечения TF-IDF-признаков из текстов воспользуйтесь классом `sklearn.feature_extraction.text.TfidfVectorizer`. 

Для предсказания целевой переменной мы будем использовать гребневую регрессию, которая реализована в классе `sklearn.linear_model.Ridge`.

Обратите внимание, что признаки `LocationNormalized` и `ContractTime` являются строковыми, и поэтому с ними нельзя работать напрямую. Такие нечисловые признаки с неупорядоченными значениями называют категориальными или номинальными. Типичный подход к их обработке - кодирование категориального признака с m возможными значениями с помощью m бинарных признаков. Каждый бинарный признак соответствует одному из возможных значений категориального признака и является индикатором того, что на данном объекте он принимает данное значение. Данный подход иногда называют one-hot-кодированием. Воспользуйтесь им, чтобы перекодировать признаки `LocationNormalized` и `ContractTime`. Он уже реализован в классе `sklearn.feature_extraction.DictVectorizer`. Пример использования:

`from sklearn.feature_extraction import DictVectorizer
enc = DictVectorizer()
X_train_categ = enc.fit_transform(data_train[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test_categ = enc.transform(data_test[['LocationNormalized', 'ContractTime']].to_dict('records'))`

Вам понадобится производить замену пропущенных значений на специальные строковые величины (например, 'nan'). Для этого подходит следующий код:

`data_train['LocationNormalized'].fillna('nan', inplace=True)
data_train['ContractTime'].fillna('nan', inplace=True)`


#### Загрузите данные об описаниях вакансий и соответствующих годовых зарплатах из файла salary-train.csv 
(либо его заархивированную версию salary-train.zip).

In [64]:
import pandas as pd
import numpy as np

df = pd.read_csv('salary-train.csv')

In [65]:
df.head(10)

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355
5,Sales and Marketing Assistant will provide adm...,Crawley,NaN,22500
6,Vacancy Ladieswear fashion Area Manager / Regi...,UK,permanent,32000
7,Reference: LR/JAN/**** Our client is one of th...,Bristol,permanent,30000
8,Sponsorship Manager London The Company A marke...,Central London,permanent,31500
9,"About Barclays Barclays moves, lends, invests ...",South East London,permanent,42499


#### Проведите предобработку
Приведите тексты к нижнему регистру (`text.lower()`).
    

In [66]:
df['FullDescription'] = df['FullDescription'].apply(lambda x: x.lower())
df.head(10)

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london ****k ****...,London,permanent,33000
1,an ideal opportunity for an individual that ha...,London,permanent,50000
2,online content and brand manager// luxury reta...,South East London,permanent,40000
3,a great local marketleader is seeking a perman...,Dereham,permanent,22500
4,registered nurse / rgn nursing home for young...,Sutton Coldfield,NaN,20355
5,sales and marketing assistant will provide adm...,Crawley,NaN,22500
6,vacancy ladieswear fashion area manager / regi...,UK,permanent,32000
7,reference: lr/jan/**** our client is one of th...,Bristol,permanent,30000
8,sponsorship manager london the company a marke...,Central London,permanent,31500
9,"about barclays barclays moves, lends, invests ...",South East London,permanent,42499


Замените все, кроме букв и цифр, на пробелы — это облегчит дальнейшее разделение текста на слова. Для такой замены в строке text подходит следующий вызов: `re.sub('[^a-zA-Z0-9]', ' ', text)`. Также можно воспользоваться методом `replace` у `DataFrame`, чтобы сразу преобразовать все тексты: `train['FullDescription'] = train['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)`.
    

In [67]:
df['FullDescription'] = df['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)
df.head(10)

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london k ...,London,permanent,33000
1,an ideal opportunity for an individual that ha...,London,permanent,50000
2,online content and brand manager luxury reta...,South East London,permanent,40000
3,a great local marketleader is seeking a perman...,Dereham,permanent,22500
4,registered nurse rgn nursing home for young...,Sutton Coldfield,NaN,20355
5,sales and marketing assistant will provide adm...,Crawley,NaN,22500
6,vacancy ladieswear fashion area manager regi...,UK,permanent,32000
7,reference lr jan our client is one of th...,Bristol,permanent,30000
8,sponsorship manager london the company a marke...,Central London,permanent,31500
9,about barclays barclays moves lends invests ...,South East London,permanent,42499


Примените `TfidfVectorizer` для преобразования текстов в векторы признаков. Оставьте только те слова, которые встречаются хотя бы в 5 объектах (параметр `min_df` у `TfidfVectorizer`).
    

In [68]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df = 5)
tfidf = vectorizer.fit_transform(df['FullDescription'])

In [69]:
tfidf

<60000x22861 sparse matrix of type '<class 'numpy.float64'>'
	with 8365759 stored elements in Compressed Sparse Row format>

Замените пропуски в столбцах `LocationNormalized` и `ContractTime` на специальную строку `'nan'`. Код для этого был приведен выше.
    

In [70]:
df['LocationNormalized'] = df['LocationNormalized'].fillna('nan')
df['ContractTime'] = df['ContractTime'].fillna('nan')
df.head(5)

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london k ...,London,permanent,33000
1,an ideal opportunity for an individual that ha...,London,permanent,50000
2,online content and brand manager luxury reta...,South East London,permanent,40000
3,a great local marketleader is seeking a perman...,Dereham,permanent,22500
4,registered nurse rgn nursing home for young...,Sutton Coldfield,nan,20355


Примените `DictVectorizer` для получения one-hot-кодирования признаков `LocationNormalized` и `ContractTime`.

In [71]:
from sklearn.feature_extraction import DictVectorizer

ln_vectorizer = DictVectorizer()
lnX = ln_vectorizer.fit_transform(df.iloc[:,1:3].T.to_dict().values())
lnX


<60000x1766 sparse matrix of type '<class 'numpy.float64'>'
	with 120000 stored elements in Compressed Sparse Row format>


Объедините все полученные признаки в одну матрицу "объекты-признаки". Обратите внимание, что матрицы для текстов и категориальных признаков являются разреженными. Для объединения их столбцов нужно воспользоваться функцией `scipy.sparse.hstack`.


In [72]:
from scipy.sparse import hstack

df2 = hstack([tfidf, lnX])

In [73]:
tfidf.shape, lnX.shape, df.shape, df.iloc[:,3:4].shape, df2.shape

((60000, 22861), (60000, 1766), (60000, 4), (60000, 1), (60000, 24627))

#### Обучите гребневую регрессию с параметрами `alpha=1` и `random_state=241`. 
Целевая переменная записана в столбце `SalaryNormalized`.


In [74]:
from sklearn.linear_model import Ridge 

regressor = Ridge(alpha=1, random_state=241)
regressor.fit(df2, df.iloc[:,3])

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=241, solver='auto', tol=0.001)

#### Постройте прогнозы для двух примеров из файла salary-test-mini.csv. 
Значения полученных прогнозов являются ответом на задание. Укажите их через пробел.

In [75]:
test = pd.read_csv('salary-test-mini.csv')
test.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,We currently have a vacancy for an HR Project ...,Milton Keynes,contract,NaN
1,A Web developer opportunity has arisen with an...,Manchester,permanent,NaN


In [76]:
test['FullDescription'] = test['FullDescription'].apply(lambda x: x.lower())
test['FullDescription'] = test['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)
tfidf = vectorizer.transform(test['FullDescription'])
test['LocationNormalized'] = test['LocationNormalized'].fillna('nan')
test['ContractTime'] = test['ContractTime'].fillna('nan')

lnX = ln_vectorizer.transform(test.iloc[:,1:3].T.to_dict().values())
test_2 = hstack([tfidf, lnX])

In [77]:
regressor.predict(test_2)

array([ 56555.61500155,  37188.32442618])